In [ ]:
!pip install adbnx-adapter
!pip install python-arango
!pip install PyYAML==5.1.1

In [ ]:
from adbnx_adapter.arangoDB_networkx_arango_adapter import ArangoDB_Networkx_Adapter

# Get temp instance
con = oasis.connect()

# Create a ArangoDB_Networkx_Adapter against the temp ArangoDB instance
ma = ArangoDB_Networkx_Adapter(conn = con)
    

In [ ]:
fraud_detection_attributes = { 'vertexCollections': {'Collection1': {'attribute1', 'attribute2'}}, 'edgeCollections' : {'Collection2' : {'att1','att2'}}

g1 = ma.create_networkx_graph(graph_name = 'FraudDetection', graph_attributes = fraud_detection_attributes)

In [ ]:
g.nodes()

In [ ]:
g.edges()